In [1]:
import pandas as pd
import datetime as dt
import fxcmpy
import psycopg2
from sqlalchemy import create_engine, inspect

# Setup the FXCM interface

In [2]:
api = fxcmpy.fxcmpy(access_token='f7d0ab4f86aafbe56492fef180f0ee8ef5d0ba1e',
                    log_level='error',
                    server='demo')

# Getting historical data and creating csv file using PostgreSQL

In [3]:
engine = create_engine('postgresql://postgres@db:5432/postgres')

for i in inspect(engine).get_table_names():
    engine.execute(f'DROP TABLE IF EXISTS "{i}";')

In [4]:
def create_csv(instr, engine):
    # Getting daily candles from September 1, 2018 to June 21, 2022
    p = 'D1'
    start = dt.datetime(2018, 9, 1)
    end = dt.datetime(2022, 6, 21)
    candles_data = api.get_candles(instrument=instr, period=p, start=start, stop=end)
    df = pd.DataFrame(candles_data)

    # Creating csv file using PostgreSQL
    if '/' in instr:
        index = instr.find('/')
        instr = instr[:index] + instr[index+1:]

    df.to_sql(f'{instr}_{p}_{start.year}_{end.year}', engine)

    conn = psycopg2.connect(host='db', dbname='postgres', user='postgres')
    try:
        with conn:
            with conn.cursor() as cur:
                db_name = inspect(engine).get_table_names()[-1]
                cur.execute(f'SELECT date, askclose FROM "{db_name}";')
                psql_df = pd.DataFrame(cur.fetchall(), columns=['date', 'askclose']).set_index('date')
                psql_df.to_csv(f'../data/{db_name}.csv')
    finally:
        conn.close()

In [5]:
create_csv('USDOLLAR', engine)
create_csv('EUR/USD', engine)